In [2]:
# from google.colab import drive

# drive.mount('/content/gdrive')
# root_path = '/D/Flatiron/Mod_5/Capstone'  #change dir to your project folder

In [3]:
# pip install pandas_datareader

In [4]:
# pip install nest_asyncio

In [5]:
# pip uninstall ipython

In [ ]:
#import Libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras import layers
import matplotlib.pyplot as plt

import praw
from psaw import PushshiftAPI
import config
import pandas as pd

import nltk
from nltk.corpus import stopwords
import string
from  nltk import FreqDist
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.collocations import *
# nltk.download('wordnet')
from nltk import word_tokenize, FreqDist

from spacy import displacy
import spacy
from tqdm import tqdm

tqdm.pandas()
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

import plotly.graph_objects as go
import plotly.express as px

import twint
import pandas as pd

import nest_asyncio 
nest_asyncio.apply()

from dateutil import rrule
import datetime as datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import time
start_time = time.time()

## Inputs:

In [1]:
currentDate = datetime.datetime.now()
q = 'TSLA'#ticker symbol
s = 'wallstreetbets' #Subreddit

NameError: name 'datetime' is not defined

In [ ]:
currentDate.date()

In [ ]:
Dates = []

for i in range(12):
    Dates.append((currentDate - relativedelta(weeks=i)).date())

In [ ]:
Dates

In [ ]:
now = datetime.datetime.now()
hundredDaysLater = now - timedelta(days=300)

for dt in rrule.rrule(rrule.DAILY, dtstart=now, until=hundredDaysLater):
    print(dt)

In [ ]:
r = praw.Reddit(
    client_id=config.reddit['client_id'],
    client_secret=config.reddit['client_secret'],
    username=config.reddit['username'],
    password=config.reddit['password'],
    user_agent='test'
)
api = PushshiftAPI(r)

## Webscraping:

In [ ]:
#Created a function to streamline webscraping
def populate(currentDate, q, subreddit, limit):
#Reddit Posts    
    Dates = []
    gen = []

    for i in range(24):#Number of months
        Dates.append((currentDate - relativedelta(months=i)).date())
    
    for i in range(len(Dates)-1):

        
        gen.extend(list(api.search_submissions(after=Dates[i+1],
                            before=Dates[i],
                            q=q,
                            subreddit='wallstreetbets',
                            filter=['created', 'title'],
                            check_for_async=False,
                            limit=limit)))
    
#Turning gen list into a pandas dataframe    
    j = []

    for i in gen:
        j.append([i.title, 
#               i.author, 
#                   i.subreddit, 
#               i.score, 
                datetime.datetime.fromtimestamp(
                int(round(i.created))
                ).strftime('%Y-%m-%d')])#Returns the time the post was created in regular time instead of UNIX time])
        
    red_posts = pd.DataFrame(j,columns=['post', 
#                                 'author', 
#                                 'subreddit', 
#                                 'score', 
                                'date'])
    
    
#Tweets dataframe
    
    c = twint.Config()

    c.Search =q #search must contain ""
    c.Min_likes = 1000 #min number of likes
    c.Until = str(currentDate.date()) #return tweets that were published before this date
    c.Since = str((currentDate - relativedelta(months=24)).date()) #return tweets published after this day(Months)
    c.Count = True
    c.Limit = 15000 # Limits the tweets to this number
    # c.Format = "Tweet id: {id} | Date: {date} | Username: {username} | Tweet: {tweet} | Mention: {mention}"
    c.Store_csv = True
    c.Output = 'Gamestop.csv'
    
    twint.run.Search(c)
    
    tweets = pd.read_csv('Gamestop.csv')
    tweets = tweets.rename(columns={"tweet": "post"})

    
    tweets = pd.concat([tweets['date'], tweets['post']], axis=1)
    
    
#combining reddit post and tweets    
    posts = pd.concat([red_posts, tweets])
    
    return posts
    

In [ ]:
posts = populate(currentDate, q, s, 300)

In [ ]:
posts

In [ ]:
posts = posts.drop_duplicates(subset=['post'], ignore_index=True)

In [ ]:
posts

## NLP:

In [ ]:
#NLP Pipeline
nlp.component_names

In [ ]:
posts['spacy'] = posts.post.progress_apply(lambda x: nlp(x))

In [ ]:
posts['sentiment'] = posts.spacy.progress_apply(lambda x: (x._.polarity))
posts['subjectivity'] = posts.spacy.progress_apply(lambda x: (x._.subjectivity))

## EDA:

In [ ]:
fig = px.scatter(posts, x='sentiment', y='subjectivity',
                hover_data=['post'])
fig.show()

In [ ]:
posts

In [ ]:
#Created two seperate dataframes for positive and negative posts
pos_list = posts[posts['sentiment'] > 0.0]
neg_list = posts[posts['sentiment'] < 0]

In [ ]:
pos_list

In [ ]:
neg_list

In [ ]:
pos_posts_concat = []
neg_posts_concat = []

for post in pos_list['post']:
    pos_posts_concat += post
for post in neg_list['post']:
    neg_posts_concat += post

In [ ]:
#created stop words list
stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``', 'link', 'tesla', 'gamestop', "'", "'s", '’', 'retard', '[', ']', 'http', 'tsla',
                  'gme','tsla', 'http', 'https', "n't", 'fuck', 'stock', 'retards', 'amc', 'buy', 'people']

In [ ]:
#function for tokenized and lemmantized the post
def process_post(posts):
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(posts)
    stopwords_removed = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stopwords_list]
    return stopwords_removed 

In [ ]:
#Fed boths postive and negative posts
pos_words = list(map(process_post, neg_list['post']))
neg_words = list(map(process_post, pos_list['post']))

In [ ]:
pos_words_list = []
for post in pos_words:
    for word in post:
        pos_words_list.append(word)
        
neg_words_list = []
for post in neg_words:
    for word in post:
        neg_words_list.append(word)

In [ ]:
pos_words_list

In [ ]:
pos_post_freqdist = FreqDist(pos_words_list)
pos_post_freqdist = pd.DataFrame(pos_post_freqdist.most_common(10), columns=['word', 'count'])

In [ ]:
#Graphing most common positive posts
fig1 = px.bar(pos_post_freqdist, y='count', x='word', 
              hover_data=['count'],
              color='count',
              title=' Most frequent positive')
fig1.show()

In [ ]:
neg_post_freqdist = FreqDist(neg_words_list)
neg_post_freqdist = pd.DataFrame(neg_post_freqdist.most_common(10), columns=['word', 'count'])

In [ ]:
#Graphing most negative posts
fig1 = px.bar(neg_post_freqdist, y='count', x='word', 
              hover_data=['count'],
              color='count',
              title=' Most frequent Negative')
fig1.show()

In [ ]:
#grouping the posts by day and then returning average values for sentiment and subjectivity
final_posts = posts.groupby(['date']).mean().sort_values(by='date', ascending=False)

In [ ]:
final_posts

In [ ]:
final_posts.index.names = ['Date']

In [ ]:
final_posts

In [ ]:
from matplotlib.pyplot import figure

In [ ]:
#Get the stock quote
tick = web.DataReader(q, data_source='yahoo', start=str((currentDate - relativedelta(months=12)).date()), end=str(currentDate.date()))
tick

In [ ]:
tick.shape

## Merging Posts:

In [ ]:
df= tick.join(final_posts)

In [ ]:
#Get number of rows and columns
df.shape

In [ ]:
df.isna()

In [ ]:
#Filled NaN values with the average of the columns
df = df.fillna(df.mean())

In [ ]:
df = pd.DataFrame(df)

In [ ]:
df['Target'] = np.append(df['Close'].iloc[1:].values, [np.nan])

In [ ]:
df

In [ ]:
df = df.dropna()

In [ ]:
df

In [ ]:
type(df)

In [ ]:
df

In [ ]:
#Visualize The data
fig,ax = plt.subplots(figsize=(20, 8))

ax.set_title('Closing Price and Sentiment History')
ax.plot(df['Close'], color='red')
ax.set_xlabel('Date', fontsize=18)
ax.set_ylabel('Closing Price USD ($)',fontsize=18)

ax2=ax.twinx()
ax2.plot(df['sentiment'], color='Blue', marker="o")
ax2.set_xlabel('Date', fontsize=18)
ax2.set_ylabel('Sentiment',fontsize=18)
ax.legend()